# python八股学习

___

## python的内存管理

### Python内存管理器
Python内存管理器负责对象内存的**分配**和**释放**。主要分为内存池分配、垃圾回收两大部分。
**内存分配器**
内存分配器是python的一个多层次系统，用于管理内存的分配和释放，目的是高效分配内存减少内存碎片化。python程序对象内存是很小的，如果为每个对象都使用操作系统内存分配malloc的话，会造成资源浪费、性能下降。因此python引入了**内存池技术**。  
python内存分配的大致步骤：
+ 程序启动之初，启动malloc，向操作系统申请一大块内存，称为area。
+ 随之将area分为多个pool，再将每个pool分为多个block。不同pool的block大小可能不一样，但相同pool下的block大小必须一致。block是存储单个python对象的基本内存单元。
+ 对于内存需求大于最大block的对象，会调用操作系统的malloc，单独存储。
+ 对于内存需求小于最大block的对象，会将其存储到大于该对象内存的最小block中。  

python的**私有堆**是一个存储python**所有对象**的数据结构，包括用户自定义对象和内建对象。

___

### 垃圾回收GC
python使用了**引用计数**和**循环垃圾回收**相结合的方法来实现内存管理。
+ 引用计数：每一个对象都有一个**引用计数器**，当对象被引用的时候，计数器会增加。引用被删除的时候，计数器会减少。当引用计数器归零的时候，该内存会被自动释放。
    + 引用计数增加行为：
        + 新建一个引用
        + 对象被作为参数传入一个函数
        + 对象被容器包含（列表与字典）--》注意容器保存的仅仅是对象的引用，因此引入深浅拷贝。
    + 引用计数减少行为：
        + 删除一个引用（"del"方法？）
        + 引用退出作用域
        + 对象被替换（重新赋值）

In [3]:
import sys

a = []
b = a
print(sys.getrefcount(a))  # 输出 3, 因为有3个引用（a、b 和 getrefcount 参数）
del b
print(sys.getrefcount(a))  # 输出 2


3
2


### 循环垃圾回收
当python的两个对象互相引用时，即使把他们都删除，他们的引用计数器都不能归零，因此引入**分代垃圾回收器**来处理循环引用的对象。
*假设：新创建的对象，相比于旧对象更容易成为垃圾，因为旧对象能存活到现在已经身经百战了*

循环垃圾回收的实现：标记、清除算法：
垃圾回收器从根节点（程序直接可访问的对象，包括全局变量、调用栈和注册的回调函数等。）对象出发，遍历由根节点直接或间接可访问的对象，并标记。分代扫描所有对象，对于未标记的（包含循环引用和未被标记的对象），直接清除。多次存活的对象可以晋级。
循环垃圾回收需要长期遍历所有对象，因此性能开销比较大。

在清除阶段检查所有对象开销太大了，因此对对象分级，级别越高的对象，在清除阶段被检查的频率越低。
+ G0:新创建的对象，垃圾回收器频繁扫描。
+ G1:由第一代晋升的对象，扫描频率低
+ G2:由第二代晋升的对象，扫描频率最低。

垃圾回收器在每一代中进行标记-清除操作，通过这种方式优化性能，减少垃圾回收的开销。

___


## help()与dir()函数
help是内置函数，查看函数和说明函数用途
dir也是内置函数，不带参数时，返回当前范围内的变量、方法和定义的类型列表；带参数时，返回参数的属性、方法

___


## Python程序退出的时候，是否会释放所有内存分配
+ 不会的。对于全局变量或一些被长期引用的对象，如果直接清除，可能导致空引用造成的不可知错误。对于循环引用，清除需要等待垃圾回收器，而垃圾回收器并不是时刻生效的，且占用资源较多，不符合迅速推出程序的需求。
+ 但是对于一些c扩展和外部库，他们或许有自己的内存回收机制，这部分内存能否顺利释放需要看模块作者是否合理设计。
+ 因此python退出时并不会主动释放内存，而是选择高效终止并推出程序，将剩余的内存占用交给操作系统清理。

___


## 什么是猴子补丁
在程序运行时动态修改某个类或模块的方法或属性，不需要改动源码。灵活性高且即使生效，但是可能会导致bug、兼容性问题等。

___


## Python的数据类型：
+ 基本数据类型：int， float， bool， complex。
+ 容器类型：
  + 字符串：文本信息
  + 列表list：有序集合，可以包含任意类型的对象，列表元素可以被修改。
  + 元组tuple：与列表类似，但内容一旦创建不可修改。
  + 字典：基于键值对的数据结构，key对value，key必须是不可变类型，值可以时任意数据类型。
    + 不可变数据类型，一旦创建不可修改，即使修改，也是重新创建新对象，int float bool complex str tuple，key为不可变主要是为了保证哈希值不变。
    + 可变数据类型例如list dict之类
  + 集合set：无序、不重复的一个集合类型，只有键，没有值。
 
___


## 为什么不要轻易给python标识符以下划线开头
因为python没有定义私有变量这种东西，因此约定下划线开头的标识符代表一个私有变量。

___


## *args的含义
*args是一种函数定义中使用的参数，用于收集所有传递给函数的剩余位置参数，并生成一个元组。
*args必须放在所有位置参数后面，*arg后面的参数必须是关键字参数。

___

位置参数与关键字参数：

位置参数：常见且默认，输入时必须按照函数定义的顺序输入。

关键字参数：调用函数的时候指定参数值，这样可以不受顺序约束。

## **kwargs的含义  
+ 用来接收任意数量的关键字参数
+ 在函数内表现为一个字典
+ 注意和*args不同，前面有两个星

In [1]:
def add_numbers(*args):
    return sum(args)

print(add_numbers(1,3,4))

def print_info(**kwargs):
    for key,value in kwargs.items():
        print(f"{key}: {value}")

print_info(name="alison",age=25)

8
name: alison
age: 25



___

## 深拷贝与浅拷贝
**浅拷贝：** 指的是把一个对象的元素的引用拷贝到另一个对象上。这意味着，如果元对象包含了对其他对象的引用，比如列表或字典，浅拷贝就只复制这些引用，而不复制对象本身。
**深拷贝：** 深拷贝创建一个新对象，然后递归地将原对象中的元素复制到新对象中。如果原对象中的元素包括对其他对象的引用，深拷贝会复制引用的对象及其整个对象图，因此新对象和原对象将不会共享任何子对象。
**如何调用深拷贝：** 深拷贝可以使用copy模块的deepcopy()函数。

___

在下面用例中，注意self.res.append(copy.deepcopy(chess))这行代码。平时处理一维list的时候，由于内容都不是容器类型，而是基本类型，因此直接复制也就相当于复制了对象。但是对于二维list，里面的对象都是容器类型，因此必须用深拷贝复制整个对象图。


In [ ]:
#
# @lc app=leetcode.cn id=51 lang=python3
#
# [51] N 皇后
#

# @lc code=start
class Solution:
    def __init__(self):
        self.res = []

    def solveNQueens(self, n: int) -> List[List[str]]:
        chess = [["." for _ in range(n)] for _ in range(n)]
        self.dfs(chess, 0, n)
        for i in self.res:
            for row in range(len(i[0])):
                i[row] = "".join(i[row])

        return self.res

    def dfs(self, chess, row, n):
        if row > n - 1:
            self.res.append(copy.deepcopy(chess))
            return
        for col in range(n):
            if self.isValid(chess, row, col, n):
                chess[row][col] = 'Q'
                self.dfs(chess, row + 1, n)
                chess[row][col] = '.'
            else: continue
        return 

    def isValid(self, chess, row, col, n):
        # 不需要判断同行是否互杀，每行只放一个
        if row > 0: # 第一行总是对的
            # 判断与之前所有行的同列是否互杀
            for i in range(row, -1, -1):
                if chess[i][col] == 'Q':
                    return False
            # 判断左上斜对角是否互杀
            for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
                if chess[i][j] == "Q":
                    return False
            # 判断右上角互杀
            for i, j in zip(range(row, -1, -1), range(col, n)):
                if chess[i][j] == "Q":
                    return False
        return True
    
# @lc code=end



## 什么是python字节码
python字节码是python代码经过编译后生成的一种低级且与平台无关的代码表示方式。该字节码可以被python解释器执行，字节码使得python运行更加高效。下例使用dis模块展示python代码对应的字节码：

In [4]:
import dis

def multiply(x, y):
    return x * y

# 使用 dis.dis() 查看函数的字节码
dis.dis(multiply)


  4           0 LOAD_FAST                0 (x)
              2 LOAD_FAST                1 (y)
              4 BINARY_MULTIPLY
              6 RETURN_VALUE


Python 在执行程序前，通常会将源代码转换成字节码（.pyc 文件）。这个过程是自动的。当你运行一个 Python 程序时，Python 解释器首先会检查是否已经存在对应的字节码版本，如果存在并且是最新的，解释器将直接执行字节码，从而跳过编译步骤，加速程序启动和运行。

___


## Cpython解释器是什么？
**Cpython**解释器是由python创始人 Guido Van Rossum 编写的python解释器。
+ 它由C语音编写，高效快速，且具有管理底层资源的能力。
+ Cpython将python代码转换为字节码，并在python虚拟机上运行
+ GIL锁：Cpython在执行字节码时使用GIL锁来确保任何时候只有一个线程在执行python字节码，简化了内存管理，但也限制了程序在多核处理器上并行执行的能力。
**其他的一些python解释器**
+ Jpython: java无锁
+ IronPython: .NET, C#
+ PyPy：有锁
+ MicroPython, 为python在微控制器和资源有限环境中使用的丐版解释器。

___


## 什么是Global Interpreter Lock？？？
官方wiki：GIL是一个互斥锁（防止两条线程同时动用进程资源，造成竞争冒险），用于保护对python对象的访问，同时防止计算一个字节码的时候的多线程操作,保护线程安全。**在GIL的作用下，python同一个进程的多个线程无法同时执行**。

python诞生于单核cpu的90年代，那是并没有考虑到多核可以同时运行多线程。这导致了即使到了多核时代，python代码也不能使用多核并行运行多线程，一次只运行一个cpu。
**Cpython的内存管理机制不是线程安全的，因此互斥锁非常有必要**
打个比方，如果三个线程都执行age=18;print(age)这两行代码，并同时执行垃圾回收线程，那么有可能垃圾回收线程会把age对象清理掉，导致一些线程无法打印出age。更严重的，导致引用计数混乱，内存泄漏。
**因此在GIL的作用下，多个线程只能分时间步运行(线程抢到gil才能运行)**
**垃圾回收回顾**
+ 引用计数
+ 标记清除
+ 分代回收

标记清除所有对象的话，非常耗费资源。因此引入分代回收机制，减少垃圾回收机制的性能开销。
**注意**
**GIL不是python的特点，是Cpython解释器的特点**
**GIL导致一个进程的多个线程无法同时运行，无法实现多核运行程序**
**GIL保证的是解释器级别的数据安全**
**写代码不需要考虑GIL，冷门知识**

## GIL的利与弊：
利：
+ 简化多线程程序编写，开发者不需要担心多线程访问对象时的线程安全问题。

弊：
性能限制，GIL限制了一次只有一个线程执行字节码。因此设计多线程密集运算程序的时候，python得不到性能提升。
但是这个问题，可以使用多进程替代多线程来解决（这样可能会占据更多资源？），或者将计算密集型运算转移到C或Cython编写的模块中。

___


## 多线程之间如何依靠GIL锁轮替?
+ 首先必须知道，在一个时间点，只有一个线程能控制解释器。
+ 在一个线程执行了一定数量的字节码或者运行一定事件后，后释放GIL锁
+ 当一个线程进入IO状态后，也会释放GIL锁。

___

## python如何实现多线程呢?
可以使用python的多线程工具包，但即使这样，也无法使用多核CPU真正进行线程并行，因为GIL锁，因此所有线程都是在cpu上轮替进行的。

___


## Python的闭包是什么?
python的闭包涉及外部函数和嵌套在内的内部函数，即使外部函数运行结束，外部函数作用域的变量依然可以由内部函数访问。闭包返回值一般是**内部函数**。
**闭包的作用：**
+ 数据隐藏： 使得某些变量变成私有变量，即，通过内部函数访问
+ 状态保持
+ 实现装饰器

In [1]:
def make_multiplier(x):
    # 外部函数定义了变量 x
    def multiplier(n):
        # 内部函数 multiplier 使用了外部函数的变量 x
        return n * x
    # 外部函数返回内部函数，这里形成了闭包
    return multiplier

# 创建一个乘以 3 的函数
times_three = make_multiplier(3)

# 使用闭包
print(times_three(10))  # 输出 30


30



___

## Python的优势是什么?
+ 易于学习
+ 完全支持面对对象编程
+ 高效的数据结构
+ 库包丰富
+ 跨平台

___


## 元组的解封装
在 Python 中，元组的解封装（tuple unpacking）是一种便捷的方式，允许你将元组中的元素直接分配到一系列变量中，而不需要通过索引访问。

In [2]:
# 定义一个元组
my_tuple = (1, 2, 3)

# 解封装元组
a, b, c = my_tuple

print(a)  # 输出 1
print(b)  # 输出 2
print(c)  # 输出 3


1
2
3


In [3]:
my_tuple = (1, 2, 3, 4, 5)
a, *b, c = my_tuple
# 带星号意思是把中间的内容赋值给一个list
print(a)  # 输出 1
print(b)  # 输出 [2, 3, 4]
print(c)  # 输出 5


1
[2, 3, 4]
5


In [4]:
my_tuple = (1, 2, 3, 4, 5)
a, _, _, _, b = my_tuple
# _代表忽略
print(a)  # 输出 1
print(b)  # 输出 5


1
5



___

## 模块
python的模块是指包含python声明和定义的文件，可以包含函数、类型、变量以及可执行代码。模块通常以.py结尾，可以被其他python代码导入和使用。
**模块的用途:**
代码重用：模块可以被多个程序或多次导入使用，避免代码重复。
命名空间隔离：模块可以帮助避免全局变量之间的冲突，因为每个模块是使用其名称空间来包含其定义的。
可维护性：模块化可以让大型程序分解成小的维护更加容易的部分。
共享服务或数据：模块可以定义函数、类和变量，这些可以被不同的程序共享使用。

**常见的内置模块与介绍：**
+ os:提供了许多操作系统服务的接口，如文件和目录访问、执行命令、处理环境变量等。
+ sys:提供对与 Python 解释器紧密相关的变量和功能的访问，例如，读写标准输入输出、操作导入的路径等。
+ math: 提供标准的算术运算函数，包括复数运算、三角函数、对数等。
+ datatime:提供处理日期和时间的类和函数
+ json:用于读取和写入 JSON 数据
+ random：随机数生成
+ collections:提供一些数据结构，比如deque， Counter

___


# 什么是python中的可哈希值：
可哈希值是可以作为字典、集合的键的值，这些值在python中大多是不可变数据类型，比如int、float、str等等。

___

# python中is和==的区别
is表示两个变量的引用地址（引用的对象）是否相同。  
== 表示两个变量引用对象的值是否相同。

___

# python中函数的参数类型有哪些？  
+ 位置参数：根据输入参数的先后顺序来解释参数的值  
+ 默认参数：含义默认值的参数
+ 可变参数：*args位置的参数，数量可变，融合为一个元组。
+ 关键字参数： **kwargs位置的参数，数量可变，输入时必须以“键-值”的形式，最终以字典的形式进入函数。

___

# [lambda x:i*x for i in range(4)]; [m(1) for m in num()]，返回的结果为什么是[3, 3, 3, 3]
lambda x:i * x实际上是一个闭包，lambda x:i*x for i in range(4)返回了四个闭包函数，并存放在列表中。闭包只有在调用时才会使用外层的局部变量i，此时i已经遍历到了3。

___

# 高阶函数：map, reduce以及filter
这三个函数形式比较相近，map(key, list)。key代表一个筛选方法，list代表输入的一个容器对象。map根据key函数，将list的内容依次变换，并返回一个可迭代对象，依次返回处理后的元素。  
fiter根据key函数的返回值（bool），来确定是否留下该数值。  
reduce会先取list的前两个值，经过key处理后返回值与list下一个值继续进行key处理，类似斐波拉契数列。

___

# __new__与__init__的区别：
__new__魔术方法用于创建实例，参数值为cls，返回值为实例对象。  

__init__魔术方法在对象创建后自动调用，根据初始化值来初始化对象的一些属性。

___

# zip()
zip()方法以两个可迭代对象作为参数，并返回由两个对象对位元素组成的元组组成的list。

___

# with语句
在python中，我们会需要短暂打开某些资源或功能，并在使用结束后释放它们，比如是文件操作、网络连接、数据库访问、神经网络前向推导时关闭自动求导等等。如果没有with语句，我们要显式的使用try，except，finally来实现对象资源的释放。
python引入了上下文管理器，使用with语句让这一切变得非常简单。with语句右边的对象，需要实现上下文管理协议，即实现__enter__和__exit__两个魔术方法，来描述进入with代码段和推出with代码段的一些操作，这两个魔术方法分别用于设置资源与释放资源，对象的生命周期得到了良好的管理。这样，进入代码块时，对象得到了妥善的加载，退出代码块时，对象也能即时释放。

___

# [i for i in range(10)] (i for i in range(20)) 分别是什么？
分别为一个列表推导式（返回列表）和生成器。 

___

# python中函数、方法的区别？方法分哪些？
函数跟类没有任何关联，一旦跟类产生丝毫瓜葛，就成为了方法。  
方法分为：
+ 实例方法， 参数名中必须要self，代表自身实例，用于传递自身属性，**只能由实例调用**。
+ 类方法，必须使用@classmethod装饰， 参数名中必须要cls，代表这个类，用于传递类的属性、方法，可以由类对象和实例对象调用。
+ 静态方法，必须使用@staticmethod装饰器，是定义在类型中的函数，一般用于处理一些数据与逻辑。 

___

# 简述单例模式与工厂模式
## 单例模式：
单例模式是一种确保一个类只有一个实例，并提供一个全局访问点来获取这个实例的设计模式。这个模式特别适合于那些全局性的资源管理，如在整个应用程序中共享的配置对象或日志对象。  

在一些比如多进程文件操作的背景下，同一个文件如果存在多个对象管理的话，可能会产生竞争冒险，导致文件数据不一致。因此我们希望某些实例只存在一个，这就产生了类型实例化的单例模式。 

具体单例模式的实现，可以实现一个singleton类型，让它继承原始类，并覆写__new__方法，只有cls._instance为False即父类实例不存在时，才实例化一个新的父类。

## 工厂模式：
工厂模式通过创建接口，避免了客户代码直接实例化某种类型，实现了代码的解耦，强化了程序的可拓展性与维护性。工厂模式的具体实现过程是，调用python的abc模块，实现产品与工厂的抽象类型或接口，然后根据接口实现具体产品与工厂，工厂用于返回产品实例，与产品一一对应。在需要产品实例的方法中，我们只需要把工厂接口作为参数传入就好了。

___

# 反射、自省
反射是python这类动态语言的一种特性，能够通过对象检查、修改自身的属性和方法。  
一些反射的功能：
+ type()，查看对象所属类型
+ dir(), 查看对象所以的方法和属性
+ getattr(), 获取对象的某个属性，字符串事件驱动
+ getattr(obj, "foo"), 如果foo是方法，会直接被执行
+ hasattr(), 获知对象是否存在某属性，字符串事件驱动
+ delattr(), 删除对象的某种属性，字符串事件驱动

___

# 元类
###python中一切皆对象，即使是类，也是对象。
python中有两种对象：  
+ 类型对象：可以被继承、实例化
+ 非类型对象：不可被继承、实例化  
python对象体系中，对象之间的两大关系：  
+ 父子关系：类型的继承，造就了对象之间的父子关系。
+ 类型实例化关系：类对象实例为实例对象。  

python中所有的类型，**都实例化自type类**。而python中所有类，都继承自object超类，即type实际上也是object类的继承。type对于object，是对象的实例化关系，也是对象的父子继承关系。type对象实例化为object对象， type对象继承自object对象，实际上以万物皆对象的观点看，这二者并不存在先有鸡先有蛋的逻辑陷阱。

## metaclass
既然万物皆对象，一切类型都是type的实例化，用户定义类型，只不过是执行了type类的__call__方法。

In [ ]:
class MyClass:
  data = 1

# 等效于
class = type(classname, superclasses, attributedict)
MyClass = type('MyClass', (), {'data': 1})

查看type类型的定义，会发现它有三个重要属性：
+ 类名
+ 父类名
+ 属性字典  
属性字典的大小是随机改动的，如果不希望类型的属性被改动，可以使用__slots__属性来指定字段，从而节省内存，但也使得类型失去了拓展性。  

了解了一切类都是type的实例化以后，就可以让自己的类不再实例化自type了，即修改类型生成的metaclass。我们可以自定义元类，比如写一个继承type类的元类，并修改__call__魔术方法，那么类型的创建过程就可以由我们自己DIY了。  

元类属于python元编程的范畴，修改了类型生成的规则，类似的元编程还有装饰器、类装饰器等等。装饰器是一个闭包，可以为原始方法提供一些启动前后的服务。


# 什么是函数式编程
python的函数式编程意味着函数可以作为参数传递（高阶函数）、作为返回值（闭包）、赋值给变量、存储在数据结构中。

# 闭包与装饰器
在函数中嵌套函数，嵌套函数中包含**对外部作用域变量的引用**，且外部函数**返回内部函数**，那么内嵌的函数称为闭包。

# 装饰器
装饰器属于元编程的内容，装饰器相当于把原始的函数作为参数封装在闭包内，覆盖原始函数的执行入口，并返回闭包函数，之后调用这个函数的时候，就可以实现一些装饰器赋予的新功能了。

# 协程（异步编程）
定义：协程是一种比多线程、多进程更轻量化的并行方式。协程是一种独立函数调用，可以**暂停或挂起执行**。非常适合IO密集的并发程序，比如蓝牙接受、网络连接等程序。  
非阻塞行为：携程需要挂起时，不会阻塞整体程序运行，会把控制权移交给其他携程程序。  

python中的一些协程：生成器（使用yield暂停函数）、使用async关键字声明的函数（使用await关键字暂停函数内部的某项操作）  

协程的优点：
+ 高效简单的并发方式，避免了多线程下操作系统与解释器锁的开支。
+ 代码直观易于理解
+ 使用范围广，比如网络IO、蓝牙IO、网络服务。

# 断言assert
断言时python一个简单的测试语句，assert关键字后面跟的时判断条件，还有判断为false时由asserterror带出的信息。